In [129]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import jieba
from jieba import lcut, analyse, Tokenizer
from wordcloud import WordCloud, STOPWORDS
from gensim.models import word2vec
from gensim.models.keyedvectors import KeyedVectors
from collections import Counter
from datetime import datetime

In [114]:
def Openfile(path):
    f = open(path, "r", encoding="utf-8")
    result = f.read()
    f.close()
    return result
def Openfile_sp(path):
    f = open(path, "r", encoding="utf-8")
    result = f.readlines()
    f.close()
    return result

In [115]:
# open file
stopwords_file = 'datafile/stop_words.txt'
jieba.set_dictionary('./dict.txt.big.txt')
jieba.load_userdict('datafile/user_dict.txt')
jieba.analyse.set_stop_words('datafile/stop_words.txt')
tigerair_file = 'datafile/tigerair_rawtext.txt'
tigerair_text = Openfile_sp(tigerair_file)
tiger_sta = 'datafile/tigerair.csv'

Building prefix dict from D:\WORK5\TechProject\TechProject2019\dict.txt.big.txt ...
Loading model from cache C:\Users\ASUS\AppData\Local\Temp\jieba.u15c153cb84f92c395787873e58f7016c.cache
Loading model cost 1.006 seconds.
Prefix dict has been built succesfully.


In [4]:
dec = Openfile_sp(stopwords_file)
stopwords = set()
for i in range(len(dec)):
    stopwords.add(dec[i].strip())


In [116]:
result = []
for i in range(len(tigerair_text)):
    r = lcut(tigerair_text[i], cut_all=False, HMM=False)
    result.append(r)

In [117]:
adj_result = []
for i in range(len(result)):
    par = []
    for j in range(len(result[i])):
        if len(result[i][j]) > 1:
            if result[i][j] not in stopwords:
                par.append(result[i][j])
    adj_result.append(par)

In [7]:
# frequency counter
c = Counter()
for i in range(len(adj_result)):
    for j in adj_result[i]:
        if len(j) > 1:
            c[j] += 1
freq_result = c.most_common(100)

df_freq = pd.DataFrame(freq_result)
df_freq.to_csv('datafile/tigerair_freqkey.csv')

In [229]:
T_model = word2vec.Word2Vec(adj_result, size=300, min_count=10)
T_model.save("model/Tigerair_model.bin")

In [230]:
# vec calculation
model = word2vec.Word2Vec.load("model/Tigerair_model.bin")
word_vectors = model.wv

In [231]:
res = word_vectors.wv.most_similar('台灣虎航', topn = 20)

D:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [232]:
res

[('加碼', 0.8564831018447876),
 ('以上', 0.8520182967185974),
 ('夏季', 0.8504301309585571),
 ('成交', 0.8249565362930298),
 ('飛航', 0.8078651428222656),
 ('冬季', 0.7879259586334229),
 ('上午', 0.786170244216919),
 ('第一波', 0.7825489044189453),
 ('現場', 0.7688313722610474),
 ('第二', 0.7672497034072876),
 ('WOW', 0.7628507614135742),
 ('顯示', 0.7624815106391907),
 ('航班', 0.7549713850021362),
 ('消息', 0.7537248134613037),
 ('原訂', 0.74970543384552),
 ('任一', 0.7493159174919128),
 ('入場', 0.7436059713363647),
 ('上述', 0.7433239817619324),
 ('即刻', 0.7408972382545471),
 ('限時', 0.7399818301200867)]

In [8]:
# vec calculation(only top 150 key result)
key_word = 'datafile/tigerair_keywords.csv'
df = pd.read_csv(key_word)
key = set()
df1 = df['key'].tolist()
for i in range(len(df1)):
    key.add(df1[i])
    
key_result = []
for i in range(len(adj_result)):
    par = []
    for j in range(len(adj_result[i])):
        if adj_result[i][j] in key:
            par.append(adj_result[i][j])
    key_result.append(par)

In [86]:
# association
key_a = '注意事項'
key_b = df1
instances = 1599
assoc_result = []
for j in range(len(key_b)):
    counter = 0
    r_counter = 0
    out_put = []
    for i1 in range(len(key_result)):
        if key_a in key_result[i1]:
            r_counter += 1
            continue
    for i2 in range(len(key_result)):
        if ((key_result[i2].count(key_a) > 0) and (key_result[i2].count(key_b[j]) > 0)):
            counter += 1
    result = counter / r_counter
    out_put.append(key_b[j])
    out_put.append(result)
    assoc_result.append(out_put)

assoc_result_adj = []
for i in range(len(assoc_result)):
    for j in range(len(assoc_result[i])):
        if assoc_result[i][1] >= 0.1:
            assoc_result_adj.append(assoc_result[i])
            break

new_assoc_result = sorted(assoc_result_adj, key=lambda I:I[1], reverse=True)
new_assoc_result


[['注意事項', 1.0],
 ['台灣虎航', 0.977116704805492],
 ['航班', 0.8009153318077803],
 ['http', 0.7780320366132724],
 ['優惠', 0.7643020594965675],
 ['訂位', 0.7551487414187643],
 ['台北', 0.7528604118993135],
 ['機場稅', 0.7391304347826086],
 ['有限', 0.7391304347826086],
 ['日期', 0.7368421052631579],
 ['說明', 0.7299771167048055],
 ['TWD', 0.7231121281464531],
 ['tigerairtw', 0.7231121281464531],
 ['優惠價', 0.7231121281464531],
 ['售完', 0.7231121281464531],
 ['期間', 0.7208237986270023],
 ['機位', 0.7070938215102975],
 ['參考', 0.7025171624713958],
 ['開放', 0.700228832951945],
 ['tw', 0.6979405034324943],
 ['使用', 0.6910755148741419],
 ['銷售', 0.6864988558352403],
 ['金額', 0.6796338672768879],
 ['適用於', 0.6773455377574371],
 ['搭乘', 0.6613272311212814],
 ['單程', 0.6567505720823799],
 ['APP', 0.6567505720823799],
 ['未稅', 0.631578947368421],
 ['系統', 0.6178489702517163],
 ['機位數', 0.6109839816933639],
 ['售價', 0.5354691075514875],
 ['購票', 0.5308924485125858],
 ['含稅價', 0.5240274599542334],
 ['訂票', 0.5148741418764302],
 ['Android'

In [74]:
instance = 1599
new = []
assoc_key = []
for i in range(len(df1)):
    if df1[i] != key_a:
        assoc_key.append(df1[i])
for keylist in range(len(assoc_key)):
    counter = 0
    result = []
    for i in range(len(key_result)):
        if (key_a and assoc_key[keylist]) in key_result[i]:
            counter += 1
    assoc = counter / instance
    result.append(assoc_key[keylist])
    result.append(assoc)
    new.append(result)    
assoc_res = pd.DataFrame(new)
new_assoc_res = df.rename(columns={'0': 'tag', '1': 'assoc'})
new_assoc_res.sort_index(by = 'value', ascending=False)
new_assoc_res


D:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)


,Unnamed: 0,key,value
0,0,台灣虎航,0.217166
1,1,TWD,0.175285
2,2,優惠,0.091663
3,3,航班,0.085327
4,4,單程,0.077609
5,5,未稅,0.074603
6,6,台北,0.074161
7,7,http,0.073135
8,8,虎迷,0.072925
9,9,活動,0.060689


In [10]:
T_test_model = word2vec.Word2Vec(key_result, size=300, min_count=10)
T_test_model.save("model/Tigerair_key_model.bin")

In [11]:
key_model = word2vec.Word2Vec.load("model/Tigerair_key_model.bin")
word_vectors = key_model.wv

In [13]:
res = word_vectors.wv.most_similar('台灣虎航', topn = 30)

D:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [14]:
res

[('中獎', 0.8075728416442871),
 ('現場', 0.7134480476379395),
 ('參加', 0.7108522653579712),
 ('留言', 0.7085999250411987),
 ('報名', 0.6894294619560242),
 ('航點', 0.6893364787101746),
 ('獲得', 0.6814465522766113),
 ('抽獎', 0.6804735064506531),
 ('號碼', 0.6745447516441345),
 ('虎迷', 0.6567276120185852),
 ('準備', 0.6308799386024475),
 ('機會', 0.617424726486206),
 ('時間', 0.6036702394485474),
 ('開頭', 0.6007678508758545),
 ('航班', 0.5992366671562195),
 ('體驗', 0.5836323499679565),
 ('小時', 0.5736051797866821),
 ('來回', 0.5725659728050232),
 ('當地', 0.5723012685775757),
 ('免費', 0.54706871509552),
 ('活動', 0.545751690864563),
 ('得獎者', 0.5387884974479675),
 ('桃園', 0.5279607772827148),
 ('機票', 0.5251080989837646),
 ('關西', 0.5212921500205994),
 ('得獎', 0.5077403783798218),
 ('異動', 0.5055073499679565),
 ('旅遊', 0.5024757385253906),
 ('起飛', 0.495614230632782),
 ('有限', 0.4816810190677643)]

In [124]:
# text analysis
product = set()
price = set()
place = set()
depart = set()
company = set()
question = set()
prize = set()


product_key = ['航班', '機票', '訂位', '機場', '搭乘', '起飛', 'depart', '出發', '機位', 
               '航線', '機位數', '航點']
price_key = ['TWD', '優惠', '單程', '未稅', '機場稅', '優惠價', '金額', '票價', '元起']
place_key = ['澳門', '東京', '日本', '沖繩', '大阪', '名古屋', '岡山', '成田', '福岡', 
             '大邱', '曼谷']
depart_key = ['台北', '高雄']
company_key = ['tigerair.com']
prize_key = ['免費', '機會', '獲得', '得獎', '得獎者', '抽獎', '中獎']
question_key = ['?', '？']


for i in range(len(product_key)):
    product.add(product_key[i])
for i in range(len(price_key)):
    price.add(price_key[i])
for i in range(len(place_key)):
    place.add(place_key[i])
for i in range(len(depart_key)):
    depart.add(depart_key[i])
for i in range(len(company_key)):
    company.add(company_key[i])
for i in range(len(question_key)):
    question.add(question_key[i])
for i in range(len(prize_key)):
    prize.add(prize_key[i])

In [166]:
output_res = []
ID = tiger_data['post_id'].tolist()
for i in range(len(result)):
    product_count = 0
    price_count = 0
    place_count = 0
    depart_count = 0
    prize_count = 0
    question_count = 0
    n_ID = ID[i]
    for j in range(len(result[i])):
        if result[i][j] in product:
            product_count += 1
            continue
        if result[i][j] in price:
            price_count += 1
            continue
        if result[i][j] in place:
            place_count += 1
            continue
        if result[i][j] in depart:
            depart_count += 1
            continue
        if result[i][j] in prize:
            prize_count += 1
            continue
        if result[i][j] in question:
            question_count += 1
            continue
        process = [n_ID, product_count, price_count, place_count, 
                   depart_count, prize_count, question_count]
    output_res.append(process)

In [168]:
res_df = pd.DataFrame(output_res)
res_df.to_csv('./Tiger_data.csv')

In [125]:
counter = 0
for i in range(len(key_result)):
    for j in range(len(key_result[i])):
        if key_result[i][j] in prize:
            counter += 1
            break
text_ana_res = counter / instances
text_ana_res



0.31207004377736086

In [123]:
counter1 = 0
for i in range(len(result)):
    for j in range(len(result[i])):
        if result[i][j] in question:
            counter1 += 1
            break
text_ana_res1 = counter1 / instances
text_ana_res1

0.6391494684177611

In [141]:
tiger_data = pd.read_csv(tiger_sta)

post_type = tiger_data['type'].tolist()
print(post_type.count('photo') / instances, 
      post_type.count('video') / instances,
      post_type.count('link') / instances)
publish = tiger_data['post_published'].tolist()
df_publish = tiger_data['post_published']
df_publish.to_csv('./T_time.csv')
adj_publish = []
for i in range(len(publish)):
    adj_time = publish[i].replace('T', ' ').replace('+0000', '')
    time = datetime.strptime(adj_time, "%Y-%m-%d %H:%M:%S")
    adj_publish.append(time)

0.6741713570981863 0.05065666041275797 0.19637273295809882


[datetime.datetime(2019, 1, 31, 12, 0),
 datetime.datetime(2019, 1, 30, 13, 1),
 datetime.datetime(2019, 1, 30, 10, 37, 12),
 datetime.datetime(2019, 1, 30, 4, 0, 1),
 datetime.datetime(2019, 1, 29, 9, 20, 18),
 datetime.datetime(2019, 1, 28, 12, 0, 1),
 datetime.datetime(2019, 1, 25, 12, 0),
 datetime.datetime(2019, 1, 25, 10, 0, 1),
 datetime.datetime(2019, 1, 24, 10, 0),
 datetime.datetime(2019, 1, 24, 4, 1, 40),
 datetime.datetime(2019, 1, 22, 12, 20, 22),
 datetime.datetime(2019, 1, 21, 10, 0, 1),
 datetime.datetime(2019, 1, 18, 10, 0, 5),
 datetime.datetime(2019, 1, 16, 10, 0, 4),
 datetime.datetime(2019, 1, 16, 4, 17, 21),
 datetime.datetime(2019, 1, 15, 10, 0),
 datetime.datetime(2019, 1, 15, 9, 0),
 datetime.datetime(2019, 1, 14, 12, 0),
 datetime.datetime(2019, 1, 14, 10, 0),
 datetime.datetime(2019, 1, 14, 2, 0),
 datetime.datetime(2019, 1, 11, 12, 0),
 datetime.datetime(2019, 1, 11, 10, 1, 51),
 datetime.datetime(2019, 1, 10, 10, 0),
 datetime.datetime(2019, 1, 9, 12, 0, 1)